In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

cuda


In [4]:
learning_rate = 0.001
training_epochs = 15
batch_size=100

In [5]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

9920512it [00:02, 4037480.40it/s]                             


Extracting MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/MNIST\raw


32768it [00:00, 170088.45it/s]           


Extracting MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw


1654784it [00:00, 2252908.91it/s]                             


Extracting MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/MNIST\raw


8192it [00:00, 45372.27it/s]            


Extracting MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw
Processing...
Done!


C:\Anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [7]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # 첫번째층
        # ImgIn shape = (?, 28, 28, 1)
        # Conv -> (?, 28, 28, 1)
        # Pool -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        
        # 두번째층
        # ImgIn shape = (?, 14, 14, 32)
        # Conv -> (?, 14, 14, 64)
        # Pool -> (?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [8]:
model = CNN().to(device)

In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device) #비용 함수에 소프트맥스 함수가 포함
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
total_batch = len(data_loader)
print(f'총 배치 수: {total_batch}')

총 배치 수: 600


In [12]:
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        # image size => (28*28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
    
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

[Epoch:    1] cost = 0.0609244332
[Epoch:    2] cost = 0.0459286794
[Epoch:    3] cost = 0.0366060361
[Epoch:    4] cost = 0.0300199483
[Epoch:    5] cost = 0.0262341741
[Epoch:    6] cost = 0.020941494
[Epoch:    7] cost = 0.0185539145
[Epoch:    8] cost = 0.0154528636
[Epoch:    9] cost = 0.0133410376
[Epoch:   10] cost = 0.0115433726
[Epoch:   11] cost = 0.00806207303
[Epoch:   12] cost = 0.00858104043
[Epoch:   13] cost = 0.00718988013
[Epoch:   14] cost = 0.00615925854
[Epoch:   15] cost = 0.00536775403


In [13]:
# 학습은 진행하지 X -> torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

Accuracy:  0.9860000014305115


C:\Anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
